In [44]:
import numpy as np
import pandas as pd
from db_connection import connect

# imports for database
import psycopg2 as pg
from db_connection import connect
from joblib import dump,load
import datetime


In [53]:
# SQL Query
sql = """
SELECT * FROM bitcoin
ORDER BY price_date DESC
LIMIT 7

"""

In [68]:
# read the raw bitcoin data from DB
df = pd.read_sql(sql=sql,con=connect())

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 13.2 on x86_64-apple-darwin, compiled by Apple clang version 11.0.3 (clang-1103.0.32.59), 64-bit',)
Connection successful


In [69]:
# drop the entry_id and date column
df.drop(['entry_id'],axis=1,inplace=True)

In [70]:
df

,price_date,open_price,high,low,close_price,volume
0,2021-12-16,48900.46,49425.57,47529.88,47665.43,27268150947
1,2021-12-15,48379.75,49473.96,46671.96,48896.72,36541828520
2,2021-12-14,46709.82,48431.40,46424.50,46612.63,34638619079
3,2021-12-13,50114.74,50205.00,45894.85,46737.48,32166727776
4,2021-12-12,49354.86,50724.87,48725.85,50098.34,21939223599
5,2021-12-11,47264.63,49458.21,46942.35,49362.51,25775869261
6,2021-12-10,47642.14,50015.25,47023.70,47243.30,30966005122


In [71]:
date = df["price_date"][0]

In [72]:
# drop price date columns
df.drop("price_date",axis=1,inplace=True)

In [73]:
# process the data into format for the model
# create new df and shifts all columns by 7 days
data = pd.DataFrame()  #create new df
for col in df.columns:
    for i in range(6, 0, -1): #range for num of days
        data[f'{col}-{i}'] = df[col].shift(i)

In [74]:
data

,open_price-6,open_price-5,open_price-4,open_price-3,open_price-2,open_price-1,high-6,high-5,high-4,high-3,...,close_price-4,close_price-3,close_price-2,close_price-1,volume-6,volume-5,volume-4,volume-3,volume-2,volume-1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,48900.46,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,47665.43,NaN,NaN,NaN,NaN,NaN,2.726815e+10
2,NaN,NaN,NaN,NaN,48900.46,48379.75,NaN,NaN,NaN,NaN,...,NaN,NaN,47665.43,48896.72,NaN,NaN,NaN,NaN,2.726815e+10,3.654183e+10
3,NaN,NaN,NaN,48900.46,48379.75,46709.82,NaN,NaN,NaN,49425.57,...,NaN,47665.43,48896.72,46612.63,NaN,NaN,NaN,2.726815e+10,3.654183e+10,3.463862e+10
4,NaN,NaN,48900.46,48379.75,46709.82,50114.74,NaN,NaN,49425.57,49473.96,...,47665.43,48896.72,46612.63,46737.48,NaN,NaN,2.726815e+10,3.654183e+10,3.463862e+10,3.216673e+10
5,NaN,48900.46,48379.75,46709.82,50114.74,49354.86,NaN,49425.57,49473.96,48431.40,...,48896.72,46612.63,46737.48,50098.34,NaN,2.726815e+10,3.654183e+10,3.463862e+10,3.216673e+10,2.193922e+10
6,48900.46,48379.75,46709.82,50114.74,49354.86,47264.63,49425.57,49473.96,48431.40,50205.00,...,46612.63,46737.48,50098.34,49362.51,2.726815e+10,3.654183e+10,3.463862e+10,3.216673e+10,2.193922e+10,2.577587e+10


In [75]:
# drop NA that are created during the shift
data.dropna(axis=0,inplace=True)

In [76]:
data

,open_price-6,open_price-5,open_price-4,open_price-3,open_price-2,open_price-1,high-6,high-5,high-4,high-3,...,close_price-4,close_price-3,close_price-2,close_price-1,volume-6,volume-5,volume-4,volume-3,volume-2,volume-1
6,48900.46,48379.75,46709.82,50114.74,49354.86,47264.63,49425.57,49473.96,48431.4,50205.0,...,46612.63,46737.48,50098.34,49362.51,2.726815e+10,3.654183e+10,3.463862e+10,3.216673e+10,2.193922e+10,2.577587e+10


In [25]:
# drop price date columns 
data.drop(["price_date-6","price_date-5","price_date-4",
           "price_date-3","price_date-2","price_date-1"],axis=1,inplace=True)

In [20]:
# read in final model file
loaded_model = load('final_model.joblib')

In [35]:
# prediction for 7 days from now 
prediction = loaded_model.predict_proba(data) # ran this 

In [84]:
#prediction[0][1]
output_message = ""
if prediction[0][0] > prediction[0][1]:
    output_message= "SELL NOW!!"
else:
    output_message = "BUY NOW!!"

    #print (pred)
    

In [85]:
output_message

'BUY NOW!!'

In [30]:
data.values

array([[4.89004600e+04, 4.83797500e+04, 4.67098200e+04, 5.01147400e+04,
        4.93548600e+04, 4.72646300e+04, 4.94255700e+04, 4.94739600e+04,
        4.84314000e+04, 5.02050000e+04, 5.07248700e+04, 4.94582100e+04,
        4.75298800e+04, 4.66719600e+04, 4.64245000e+04, 4.58948500e+04,
        4.87258500e+04, 4.69423500e+04, 4.76654300e+04, 4.88967200e+04,
        4.66126300e+04, 4.67374800e+04, 5.00983400e+04, 4.93625100e+04,
        2.72681509e+10, 3.65418285e+10, 3.46386191e+10, 3.21667278e+10,
        2.19392236e+10, 2.57758693e+10]])

In [13]:
# read pickle file

# run predict function (output 2x1)

# 

""
0
1
2
3
4
5


In [5]:
# read in your final model

In [ ]:
# run the data through your model to spit out a prediction

In [30]:
class Sentence:
    def __init__(self,sentence):
        self.sentence = sentence
        self.index = 0
        self.words = self.sentence.split()
        
    def __iter__(self):
        return self
        
    def __next__(self):
        if self.index >= len(self.words):
            raise StopIteration
        index = self.index
        self.index += 1
        return self.words[index]


In [31]:
my_sentence = Sentence("This is a Test")

In [32]:
my_sentence

In [33]:
for word in my_sentence:
    print(word)

This
is
a
Test


In [55]:
def sentences(sentence):
    for word in sentence.split():
        yield word


In [56]:
my_sentence = sentences("This is a Test")

In [57]:
for word in my_sentence:
    print(word)

This
is
a
Test


In [47]:
datetime.datetime

TypeError: function missing required argument 'year' (pos 1)

In [78]:
predicted_date = date

In [80]:
predicted_date += datetime.timedelta(days=7)

In [81]:
predicted_date

datetime.date(2021, 12, 23)

In [51]:
print(df["price_date"][0])

2021-12-23


In [83]:
print(f"Today's date is {date}")

Today's date is 2021-12-16
